In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [3]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]

In [4]:
test_data = pd.read_csv('data/technical/test.csv')
shops_in_test_data = set()

for i, row in test_data.iterrows():
    shops_in_test_data.add(row['shop_id'])

all_shops = set([*range(0, 60)])
shops_not_present = all_shops-shops_in_test_data
shops_not_present

for missing_shop in shops_not_present:
    index_names = data[ data['shop_id'] == missing_shop ].index
    data.drop(index_names, inplace=True)

In [5]:
X = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y = data[data.date_block_num < 34]['item_cnt_month']
x = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.72, random_state=42)


parameter_candidates = [{'alpha': [1.9, 2.0, 3, 5],
                         'solver': ['auto', 'sparse_cg', 'saga']
                         }]
model = Ridge()

cv_model = GridSearchCV(estimator=model,
                        param_grid=parameter_candidates,
                        n_jobs=3)

cv_model.fit(X_train, y_train)

print(cv_model.best_params_)

{'alpha': 5, 'solver': 'auto'}


In [7]:
model = Ridge(alpha=1.5)
score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
print('RMSE %2f' %(-1 * score.mean()))

RMSE 0.814322
